In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
from harpia.filters import filters as filters_H
from skimage.filters import prewitt,sobel,gaussian,unsharp_mask
from skimage.restoration import non_local_means
from scipy import ndimage

# Function to check if two circles overlap
def circles_overlap(x1, y1, r1, x2, y2, r2):
    return (x2 - x1) ** 2 + (y2 - y1) ** 2 < (r1 + r2) ** 2

def create_circles(width =2048 , height =2048, num_circles = 1000, min_radius = 40, max_radius = 50):
    circles = []
    image = np.zeros((height, width), dtype=np.int32)
    for _ in range(num_circles):
        # Random position
        count = 0
        while True:
            x = np.random.randint(max_radius, width - max_radius)
            y = np.random.randint(max_radius, height - max_radius)
        
            # Check if the new circle overlaps with any existing circle
            #overlap = False
            #for circle in circles:
            #    if circles_overlap(x, y, max_radius, circle[0], circle[1], circle[2]):
            #        overlap = True
            #        break
        
            #if not overlap:
            #    break

            if count>num_circles:
                break

            count = count +1
    
        # Random radius and intensity
        radius = np.random.randint(min_radius, max_radius)
          # Signed integer intensity
    
        # Draw the circle on the image
        yy, xx = np.ogrid[:height, :width]
        circle_mask = (xx - x) ** 2 + (yy - y) ** 2 <= radius ** 2
        image[circle_mask] = 1
    
        # Save circle information for future checks
        circles.append((x, y, radius))
    return np.random.normal(image,scale=0)

def contiguous(array: np.ndarray ) -> np.ndarray:
    if array.dtype != np.int32:
        array = np.ascontiguousarray(array.astype(np.int32))
    elif not array.flags['C_CONTIGUOUS']:
        array = np.ascontiguousarray(array)
    return array

#z,y,x = 1,2048,2048
#img2d = create_circles(width=x,height=y)
#img = contiguous((np.stack([img2d] * z, axis=0)).reshape(z,y,x))

#z,x,y = 1,207,190
#path ='/home/ricardo.grangeiro/Desktop/crua_A_190x207x100_16b.raw'
#img_raw = np.fromfile(path,dtype=np.uint16).reshape(100,207,190)
#img = contiguous((np.stack([img_raw[0]] * z, axis=0)).reshape(z,x,y))


#import numpy as np
#import matplotlib.pyplot as plt

# Path to the raw file
path = '/home/ricardo.grangeiro/Desktop/Recon_fdk__tomo_z1_930_z1z2_1000_esp_1264_expt_1s_000_1540x1540x1000_float32.raw'

# Dimensions of the 3D volume
x, y, z = 1540, 1540, 1000  # Corrected order based on your description

# Specify the slice index (e.g., middle slice along z-axis)
slice_index = 300

# Memory-map the file and access the slice directly
# The shape is (y_dim, x_dim, z_dim) and dtype is float32
F = np.memmap(path, dtype=np.float32, mode='r', shape=(z, x, y))

# Extract the slice you want (here along the z-axis)
img = contiguous((np.stack([F[slice_index]] * 1, axis=0)).reshape(1,x,y))

# Display the slice
plt.imshow(img[0], cmap='gray')
plt.colorbar()
plt.title(f'Slice at z = {slice_index}')
plt.show()

z=1

### Sobel filter

In [ ]:
out = np.ascontiguousarray(np.zeros_like(img).astype(np.float32))

filters_H.sobel(img.astype(np.int16),out,x,y,z,0)
out = out/out.max()

y_gt = sobel(img[0])
y_gt = y_gt/y_gt.max()

fig = plt.figure(figsize=(16,16))

plt.subplot(331)
plt.title('Original image')
plt.colorbar(plt.imshow(img[0],cmap='gray'))
plt.axis('off')

plt.subplot(332)
plt.title('Annotat3d-Sobel Filter')
plt.colorbar(plt.imshow(out[0],cmap='gray'))
plt.axis('off')

plt.subplot(333)
plt.title('Skimage-Sobel Filter')
plt.colorbar(plt.imshow(y_gt,cmap='gray'))
plt.axis('off')

plt.tight_layout()

error = np.abs(y_gt-out[0]).sum()/(y_gt.size)
error = np.round(error,3)

print(error)

### Prewitt filter

In [ ]:
out = np.ascontiguousarray(np.zeros_like(img).astype(np.float32))

filters_H.prewitt(img,out,x,y,z,0)
out = out/out.max()

y_gt = prewitt(img[0])
y_gt = y_gt/y_gt.max()

fig = plt.figure(figsize=(16,16))

plt.subplot(331)
plt.title('Original image')
plt.colorbar(plt.imshow(img[0],cmap='gray'))
plt.axis('off')

plt.subplot(332)
plt.title('Annotat3d-Prewitt Filter')
plt.colorbar(plt.imshow(out[0],cmap='gray'))
plt.axis('off')

plt.subplot(333)
plt.title('Skimage-Prewitt Filter')
plt.colorbar(plt.imshow(y_gt,cmap='gray'))
plt.axis('off')

plt.tight_layout()

error = np.abs(y_gt-out[0]).sum()/(y_gt.size)
error = np.round(error,3)

print(error)

### Unsharp Mask

In [ ]:
out = np.ascontiguousarray(np.zeros_like(img).astype(np.float32))

filters_H.unsharp_mask(img,out,x,y,z,1,10,0,0)

y_gt = unsharp_mask(img[0],1,10,preserve_range=True)

out = out/out.max()
y_gt = y_gt/y_gt.max()

fig = plt.figure(figsize=(16,16))
plt.subplot(331)
plt.title('Original image')
plt.colorbar(plt.imshow(img[0],cmap='gray'))
plt.axis('off')

plt.subplot(332)
plt.title('Annotat3d- Unsharp Mask Filter')
plt.colorbar(plt.imshow(out[0],cmap='gray'))
plt.axis('off')

plt.subplot(333)
plt.title('Skimage-Unsharp Mask Filter')
plt.colorbar(plt.imshow(y_gt,cmap='gray'))
plt.axis('off')

plt.tight_layout()

error = np.abs(y_gt-out[0]).sum()/(y_gt.size)
error = np.round(error,3)

print(error)

### Gaussian filter

In [ ]:
out = np.ascontiguousarray(np.zeros_like(img).astype(np.float32))
filters_H.gaussian(img,out,x,y,z,1,0)

y_gt = gaussian(img[0],1,mode='reflect',preserve_range=True)

out = out/out.max()
y_gt = y_gt/y_gt.max()

fig = plt.figure(figsize=(16,16))
plt.subplot(331)
plt.title('Original image')
plt.colorbar(plt.imshow(img[0],cmap='gray'))
plt.axis('off')

plt.subplot(332)
plt.title('Annotat3d-Gaussian Filter')
plt.colorbar(plt.imshow(out[0],cmap='gray'))
plt.axis('off')

plt.subplot(333)
plt.title('Skimage-Gaussian Filter')
plt.colorbar(plt.imshow(y_gt,cmap='gray'))
plt.axis('off')

plt.tight_layout()

error = np.abs(y_gt-out[0]).sum()/(y_gt.size)
error = np.round(error,3)

print(error)

### Median filter

In [ ]:
from skimage.filters.rank import median
out = np.ascontiguousarray(np.zeros_like(img).astype(np.int32))

filters_H.median(img,out,x,y,z,3,3,1)

y_gt = median(img[0].astype(np.uint16))

fig = plt.figure(figsize=(16,16))
plt.subplot(331)
plt.title('Original image')
plt.colorbar(plt.imshow(img[0],cmap='gray'))
plt.axis('off')

plt.subplot(332)
plt.title('Annotat3d-Median Filter')
plt.colorbar(plt.imshow(out[0],cmap='gray'))
plt.axis('off')

plt.subplot(333)
plt.title('Skimage-Median Filter')
plt.colorbar(plt.imshow(y_gt,cmap='gray'))
plt.axis('off')

plt.tight_layout()

error = np.abs(y_gt-out[0]).sum()/(y_gt.size)
error = np.round(error,3)

print(error)

### Mean filter

In [ ]:
from scipy import ndimage
img = contiguous((np.stack([img[0]] * z, axis=0)).reshape(z,x,y))
out = np.ascontiguousarray(np.zeros_like(img).astype(np.float32))

filters_H.mean(img,out,x,y,z,11,11,1)

k = np.full((11,11),1.0)

y_gt = ndimage.convolve(img[0],k)/(11*11)


fig = plt.figure(figsize=(16,16))
plt.subplot(331)
plt.title('Original image')
plt.colorbar(plt.imshow(img[0],cmap='gray'))
plt.axis('off')

plt.subplot(332)
plt.title('Annotat3d-Mean Filter')
plt.colorbar(plt.imshow(out[0],cmap='gray'))
plt.axis('off')

plt.subplot(333)
plt.title('Skimage-Mean Filter')
plt.colorbar(plt.imshow(y_gt,cmap='gray'))
plt.axis('off')

plt.tight_layout()

error = np.abs(y_gt-out[0]).sum()/(y_gt.size)
error = np.round(error,3)

print(error)

### Non-Local Means filter

In [ ]:
from skimage.restoration import denoise_nl_means

h=500
small_window=3
big_window=51

img = contiguous((np.stack([img[0]] * z, axis=0)).reshape(z,x,y))
out = np.ascontiguousarray(np.zeros_like(img).astype(np.float64))

filters_H.non_local_means(img[0],out[0],x,y,small_window,big_window,500,0)

#y_gt = denoise_nl_means(img[0],h=h,patch_size=5,patch_distance=21,preserve_range=True)

out = out/out.max()
y_gt =  y_gt/y_gt.max()

fig = plt.figure(figsize=(16,16))
plt.subplot(331)
plt.title('Original image')
plt.colorbar(plt.imshow(img[0],cmap='gray'))
plt.axis('off')

plt.subplot(332)
plt.title('Annotat3d-Nl means Filter')
plt.colorbar(plt.imshow(out[0],cmap='gray'))
plt.axis('off')

plt.subplot(333)
plt.title('Skimage-Nl means Filter')
plt.colorbar(plt.imshow(y_gt,cmap='gray'))
plt.axis('off')

plt.tight_layout()

error = np.abs(y_gt-out[0]).sum()/(y_gt.size)
error = np.round(error,3)

print(error)

### Canny filter

In [ ]:
from skimage import feature
img = contiguous((np.stack([img[0]] * z, axis=0)).reshape(z,x,y))
out = np.ascontiguousarray(np.zeros_like(img).astype(np.float32))

filters_H.canny(img,out,x,y,z,1.5,1,700)

y_gt = feature.canny(img[0],sigma=1.5,low_threshold=1,high_threshold=700,mode='reflect')

fig = plt.figure(figsize=(16,16))
plt.subplot(331)
plt.title('Original image')
plt.colorbar(plt.imshow(img[0],cmap='gray'))
plt.axis('off')

plt.subplot(332)
plt.title('Annotat3d-Canny Filter')
plt.colorbar(plt.imshow(out[0],cmap='gray'))
plt.axis('off')

plt.subplot(333)
plt.title('Skimage-canny Filter')
plt.colorbar(plt.imshow(y_gt,cmap='gray'))
plt.axis('off')

plt.tight_layout()

error = np.abs(y_gt-out[0]).sum()/(y_gt.size)
error = np.round(error,3)

print(error)